# Bias in Hate Speech Detection – Von einfachen Modellen zu Counterfactual Fairness

Dieses Notebook kombiniert Elemente aus zwei Tutorials:

- https://www.kaggle.com/alexisbcook/identifying-bias-in-ai
- https://github.com/biaslyze-dev/biaslyze

Ziel ist es, die Entstehung und Analyse von Bias in Hate Speech Detection Modellen nachzuvollziehen.
Dazu starten wir mit einem sehr einfachen Modell und steigern schrittweise die Komplexität, bis wir schließlich systematisch Counterfactual Fairness messen.

# 1. Einführung: Bias in Toxicity Detection

Am Ende des Jahres 2017 stellte die Plattform Civil Comments den Betrieb ein und veröffentlichte ca. 2 Millionen moderierte Kommentare. Jigsaw unterstützte die Annotation der Daten, und 2019 organisierte Kaggle die Challenge "Unintended Bias in Toxicity Classification"

Diese Daten eignen sich gut, um zu untersuchen, wie Modelle für Hate Speech Detection Verzerrungen entwickeln können - oft ohne, dass dies beabsichtigt ist.
Wir beginnen mit einem stark vereinfachten Setup, um grundlegende Muster sichtbar zu machen.

The code cell below loads some of the data from the competition.  We'll work with thousands of comments, where each comment is labeled as either "toxic" or "not toxic".

Begin by running the next code cell.  
- Clicking inside the code cell.
- Click on the triangle (in the shape of a "Play button") that appears to the left of the code cell.

The code will run for approximately 30 seconds.  When it finishes, you should see as output a message saying that the data was successfully loaded, along with two examples of comments: one is toxic, and the other is not.

# 1.2 Einstieg: Einfaches Modell basierend auf Worthäufigkeiten

Wir laden die Daten, wählen ein Subset und repräsentieren die Texte über reine Wortzählungen. Durch diese bewusst simple Repräsentation werden früh erste Hinweise auf Bias sichtbar.

Die folgenden Schritte laden und vektorisieren die Daten und trainieren ein logistisches Regressionsmodell.

In [ ]:
# TODO logistisches Regressionsmodell

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

# Get the same results each time
np.random.seed(0)

# Load the training data
data = pd.read_csv("data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

# Break into training and test sets
comments_train, comments_test, y_train, y_test = train_test_split(comments, target, test_size=0.30, stratify=target)

# Get vocabulary from training data
vectorizer = CountVectorizer()
vectorizer.fit(comments_train)

# Get word counts for training and test sets
X_train = vectorizer.transform(comments_train)
X_test = vectorizer.transform(comments_test)

# Preview the dataset
print("Data successfully loaded!\n")
print("Sample toxic comment:", comments_train.iloc[22])
print("Sample not-toxic comment:", comments_train.iloc[17])

In [ ]:
# first 10 entries of the dataset
data[0:10]

In [ ]:
# Take a look at the dataset, e.g how many samples are there that contain the word 'black' that are labled as toxic

len(data.loc[(data['comment_text'].str.contains("black", case=False, na=False)) & (data['target'] >= 0.7)])

# 1.3 Modelltraining und erste Evaluation

Das folgende Code-Snippet trainiert ein sehr einfaches Modell. Es verwendet CountVectorizer (zählt Häufigkeit von Wörtern im comment_text) und Logistic Regression. Anschließend zeigt es die Testgenauigkeit.

In [ ]:
from sklearn.linear_model import LogisticRegression
import joblib

# Train a model and evaluate performance on test dataset
#classifier = LogisticRegression(max_iter=2000)
#classifier.fit(X_train, y_train)

# load already trained model
classifier = joblib.load('logistic_model_vectorizer.pkl')
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

# Function to classify any string
def classify_string(string, investigate=False):
    prediction = classifier.predict(vectorizer.transform([string]))[0]
    if prediction == 0:
        print("NOT TOXIC:", string)
    else:
        print("TOXIC:", string)

Rund 93% Genauigkeit wirken zunächst überzeugend – doch diese Zahl sagt nichts über mögliche Ungleichbehandlung/ Diskriminierung aus.

# 1.4 Erste Exploration: Wie reagiert das Modell auf eigene Beispiele?

Hier kannst du eigene Kommentare testen und beobachten, wie das Modell sie einordnet.

Die Funktion classify_string sorgt für die Ausgabe, z. B.:


In [ ]:
# Comment to pass through the model
my_comment = "I have a black friend"

# Do not change the code below
classify_string(my_comment)

# 1.5 Feature-Gewichte: Welche Wörter gelten als besonders toxisch?

Das Modell weist jedem vorkommenden Wort einen Koeffizienten zu. Hohe Werte bedeuten „toxisch“, niedrige „nicht toxisch“.

Die folgende Codezelle listet die 10 am stärksten positiv gewichteten Wörter:

In [ ]:
coefficients = pd.DataFrame({"word": sorted(list(vectorizer.vocabulary_.keys())), "coeff": classifier.coef_[0]})
print(coefficients.sort_values(by=['coeff']).tail(10))

Schaut euch die 'toxischsten' Wörter der Zelle oberhalb an. Seid ihr überrascht? Gibt es Wörter die nicht in der Liste sein sollten? 

# 1.6 Sensitivität gegenüber Gruppenbegriffen

In diesem Abschnitt werden minimal veränderte Sätze eingegeben, um zu sehen, wie stark das Modell auf Begriffe reagiert wie:

- muslim / christian
- white / black

Dies ist ein klassisches Setup zur Erkennung von bias-sensitiven Token.

In [ ]:
# Set the value of new_comment
new_comment = "I have a white friend"

# Do not change the code below
classify_string(new_comment)
coefficients[coefficients.word.isin(new_comment.split())]

Es wird sichtbar, dass das Modell teilweise unterschiedliche Gewichte für ähnliche Begriffe verwendet, obwohl der Kontext derselbe ist.

# 1.7 Reflexion: Erste Erklärung zu Bias-Effekten

Warum passiert das?

Ursachen können u. a. sein:

- Ungleichverteilung sensibler Begriffe in den Trainingsdaten
- Häufige Korrelation zwischen Minderheitengruppen und toxischen Kontexten
- Einfaches zählen von Worthäufigkeiten ignoriert Kontext
- Einzelne Begriffe werden überinterpretiert


# 2. Etwas komplexere Repräsentation und größere Datenbasis

Jetzt wechseln wir auf einen größeren Datensatz und nutzen TF-IDF statt roher Frequenzen.
Das Ziel: Eine robustere Repräsentation, die häufige Wörter weniger stark gewichtet.

Wir verwenden:

- TF-IDF Vectorizer()
- Logistic Regression
- deutlich größeren Datensatz (226.235 Kommentare)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

In [ ]:
# load data
df = pd.read_csv("train.csv"); df.tail()
# all that is labeled as toxis, severe_toxic, obscene... gets labeled as toxic
df["target"] = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(axis=1) > 0
df.columns

In [ ]:
# Have a look at the dataset

# filters data where both target == 1 -> labeled as toxic and the comment text inludes a certain word: here 'old'
len(df.loc[(df['comment_text'].str.contains("old", case=False, na=False)) & (df['target'] == 1)]['comment_text'])


# 2.1 Bias-Analyse mit Biaslyze

Nun führen wir eine systematische Analyse durch.
Biaslyze nutzt counterfactual token fairness:

Wie stark verändert sich die Modell-Prediktion, wenn ein sensitives Wort durch ein anderes aus derselben Kategorie ersetzt wird?

Beispiele für Konzepte:

- religion
- gender
- ethnicity


In [ ]:
# train model
#clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=30000, stop_words="english"), LogisticRegression(C=10))
#clf.fit(df.comment_text, df.target)

#load already trained model
clf = joblib.load("Tfidf_Regression_jigsaw.pkl")

In [ ]:
# Try some examples

if clf.predict(['I have a gay friend'])[0] == True:
    print("TOXIC")
else:
    print('NOT TOXIC')

In [ ]:
from biaslyze.bias_detectors import CounterfactualBiasDetector
from biaslyze.concept_class import Concept

bias_detector = CounterfactualBiasDetector()

In [ ]:
### OPTIONAL ####

# lets have a look at the Concepts 

'''Example Concept:  "religion": [
        {"keyword": "jew", "function": ["NOUN"], "category": "judaism"},
        {"keyword": "jewish", "function": ["ADJ"], "category": "judaism"},
        {"keyword": "jews", "function": ["PROPN", "NOUN"], "category": "judaism"},
        {"keyword": "judaism", "function": ["NOUN", "PROPN"], "category": "judaism"},
        {
            "keyword": "muslim",
            "function": ["NOUN", "ADJ"],
            "category": "islam",
        },
        {"keyword": "muslims", "function": ["NOUN", "PROPN"], "category": "islam"},
        {"keyword": "moslem", "function": ["NOUN", "PROPN"], "category": "islam"},
        {"keyword": "moslems", "function": ["NOUN", "PROPN"], "category": "islam"},
        {"keyword": "islam", "function": ["NOUN", "PROPN"], "category": "islam"},
        {
            "keyword": "christ",
            "function": ["NOUN", "PROPN"],
            "category": "christianity",
        },
        {
            "keyword": "christian",
            "function": ["NOUN", "PROPN"],
            "category": "christianity",
        },
        {
            "keyword": "christianity",
            "function": ["NOUN", "PROPN"],
            "category": "christianity",
        },
        {
            "keyword": "christians",
            "function": ["NOUN", "PROPN"],
            "category": "christianity",
        },
        {"keyword": "buddhism", "function": ["NOUN", "PROPN"], "category": "buddhism"},
        {"keyword": "buddhist", "function": ["NOUN", "PROPN"], "category": "buddhism"},
        {"keyword": "buddhists", "function": ["NOUN", "PROPN"], "category": "buddhism"},
        {"keyword": "hindu", "function": ["NOUN"], "category": "hinduism"},
        {"keyword": "hindus", "function": ["NOUN"], "category": "hinduism"},
        {"keyword": "hinduism", "function": ["NOUN", "PROPN"], "category": "hinduism"},
    ]'''

# how to add new concepts in the same manner

names_concept = Concept.from_dict_keyword_list(
    name="names1",
    lang="en",
    keywords=[{"keyword": "Hans", "function": ["NOUN"]}],
)

bias_detector.register_concept(names_concept)

# add concept to 'concepts_to_consider' bias_detector.process()

In [ ]:
counterfactual_detection_results = bias_detector.process(
    texts=df.comment_text.sample(1000, random_state=42),
    labels=df.target.tolist(),
    predict_func=clf.predict_proba,
    concepts_to_consider=["religion", "gender", "ethnicity", "names1"],
    max_counterfactual_samples=None,
)

# 2.2 Interpretation der Counterfactual Scores

Biaslyze identifiziert:

- welche Begriffe keinen Einfluss haben (omitted keywords)
- welche Begriffe starken Einfluss haben (Counterfactual Score)

Ein hoher Score bedeutet: Das Modell reagiert stark auf den Begriff, unabhängig vom restlichen Kontext

Für sensible Konzepte gibt dies wertvolle Hinweise:

- Welche Begriffe triggern stark?
- Welche Begriffe sollten überprüft werden?
- Welche verzerrte Gewichtung könnte das Modell gelernt haben?

In [ ]:
# omitted keywords
print(counterfactual_detection_results.concept_results[3].omitted_keywords)

# 2.3 Plotten der Ergebnisse

Der couterfactual score wird definiert als die Differenz zwischen dem vorhergesagten Wahrscheinlichkeitswert für den couterfactual Text und  dem vorhergesagten Wahrscheinlichkeitswert für den Originaltext.
    
> couterfactual_score = P(toxisch | couterfactual_Text) - P(toxisch | Originaltext)

Je weiter also die Punktzahl einer Probe von Null entfernt ist, desto größer ist die Änderung in der Entscheidung des Modells, ob ein Kommentar toxisch oder nicht toxisch ist, wenn er durch dieses Schlüsselwort ersetzt wird. In diesem Fall ist die positive Klasse „toxisch” und die negative Klasse „nicht toxisch”. Wie ihr seht, führt das Ersetzen eines beliebigen anderen geschlechtsspezifischen Schlüsselworts durch das Wort „Mutter” dazu, dass die Klassifizierung der Proben mit größerer Wahrscheinlichkeit als „toxisch” eingestuft wird.

In [ ]:
concept = "gender"
top_n = 20 


### Ignore the following code - only plots the data 

def _plot_box_plot(dataf: pd.DataFrame, top_n: int = None):
    """Plot a box plot of scores.

    Args:
        dataf: A dataframe with scores for each sample.
        top_n: Only plot the top n concepts.

    Returns:
        A matplotlib axis.
    """
    # sort the dataframe by median absolute value
    sort_index = dataf.median().abs().sort_values(ascending=True)
    sorted_dataf = dataf[sort_index.index]
    if top_n:
        sorted_dataf = sorted_dataf.iloc[:, -top_n:]
    ax = sorted_dataf.plot.box(
        vert=False, figsize=(12, int(sorted_dataf.shape[1] / 2.2))
    )
    ax.vlines(
        x=0,
        ymin=0.5,
        ymax=sorted_dataf.shape[1] + 0.5,
        colors="black",
        linestyles="dashed",
        alpha=0.5,
    )
    return ax

dataf = counterfactual_detection_results._get_result_by_concept(concept=concept)
ax = _plot_box_plot(dataf, top_n=top_n)
ax.set_title(
            f"Distribution of counterfactual scores for concept '{concept}'\nsorted by median score"
        )
ax.set_xlabel(
            "Counterfactual scores - differences from zero indicate the direction of bias."
        )
plt.show()

In [ ]:
# only works with jupyter running on a local machine
# besides counterfactual score, ksr, sample based histograms and the augmented textsamples can be explored 

counterfactual_detection_results._get_counterfactual_samples_by_concept(concept="religion")
counterfactual_detection_results.dashboard(num_keywords= 10, port = 8090)